In [ ]:
from datasets import load_dataset
import pandas as pd 
from sklearn.model_selection import train_test_split

In [ ]:
# Load directly from Hugging Face Hub by its dataset identifier
hate_explain = load_dataset("Abhi0072/HateXplain")

# convert to dataframe
hate_explain = hate_explain['train'].to_pandas()

In [ ]:
hate_explain.info()
hate_explain.head(10)

In [ ]:
label_mapping = {
    "normal": 2,
    "hatespeech": 0,
    "offensive": 1

}

# Apply the mapping to the label column
hate_explain['label'] = hate_explain['label'].apply(lambda label: label_mapping.get(label))

In [ ]:
hate_explain.head(10)

In [ ]:
davidson_dataset = load_dataset("tdavidson/hate_speech_offensive")
# convert to dataframe
davidson_df = davidson_dataset['train'].to_pandas()

In [ ]:
# Rename 'class' column to 'label'
davidson_df = davidson_df.rename(columns={'class': 'label', 'tweet': 'text'})
davidson_df = davidson_df.drop(columns=["neither_count", "offensive_language_count", "hate_speech_count","count"])

In [ ]:
davidson_df.info()

In [ ]:
gab = load_dataset("juliadollis/The_Gab_Hate_Corpus_ghc_train_original")
# convert to dataframe
gab = gab['train'].to_pandas()

# Filter hate speech (either attacks dignity or calls for violence)
gab_hate = gab[(gab["hd"] == 1) | (gab["cv"] == 1)].copy()

gab_hate['label'] = 0
gab_hate = gab_hate.drop(columns=["hd", "cv", "vo"])


# Filter only offensive speech (vulgar but not hateful)
gab_offensive = gab[(gab["vo"] == 1) & (gab["hd"] == 0) & (gab["cv"] == 0)].copy()

gab_offensive['label'] = 1
gab_offensive = gab_offensive.drop(columns=["hd", "cv", "vo"])
gab_offensive.head()

gab = pd.concat([gab_hate, gab_offensive])

gab.info()


In [ ]:
frankshu = load_dataset("thefrankhsu/hate_speech_twitter")
# convert to dataframe
frankshu = frankshu['train'].to_pandas()

frankshu_hate = frankshu[frankshu['label'] == 1].copy()
frankshu_hate['label'] = 0

frankshu_normal = frankshu[frankshu['label'] == 0].copy()
frankshu_normal['label'] = 2

frankshu = pd.concat([frankshu_hate, frankshu_normal])

frankshu = frankshu.drop(columns=['categories'])
frankshu = frankshu.rename(columns={"tweet": "text"})
frankshu.info()

In [ ]:
frankshu = load_dataset("")

In [ ]:
dataset = pd.concat([hate_explain, davidson_df, gab, frankshu])

In [ ]:
dataset.info()

In [ ]:
dataset["label"].value_counts()

In [ ]:
print(dataset.iloc[:10]['text'])

In [ ]:
train_data = pd.read_csv("../data/processed_training_data.csv")

In [ ]:
train_data['label'].value_counts()